# 🗄️ 步骤 4：导入到 Neo4j

将知识图谱导入 Neo4j 数据库。

**前提**: 配置好 Neo4j 连接（推荐使用 [Neo4j Aura Free](https://neo4j.com/cloud/aura-free/)）

In [ ]:
import os, sys, json, yaml, logging
import pandas as pd
from tqdm import tqdm
from neo4j import GraphDatabase
from datetime import datetime

# 加载配置
PROJECT_ROOT = os.environ.get('PROJECT_ROOT', '/content/PWD_Project')
with open(f'{PROJECT_ROOT}/config/config_colab.yaml', 'r') as f:
    CONFIG = yaml.safe_load(f)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('Neo4j')
print(f"✅ Neo4j URI: {CONFIG['neo4j']['uri']}")

In [ ]:
# 加载数据
output_dir = CONFIG['output']['base_directory']
concepts_df = pd.read_csv(f"{output_dir}/concepts_final.csv")
relationships_df = pd.read_csv(f"{output_dir}/relationships_final.csv")
print(f"✅ 概念: {len(concepts_df)}, 关系: {len(relationships_df)}")

In [ ]:
# 连接 Neo4j
driver = GraphDatabase.driver(
    CONFIG['neo4j']['uri'],
    auth=(CONFIG['neo4j']['user'], CONFIG['neo4j']['password'])
)
driver.verify_connectivity()
print("✅ Neo4j 连接成功")

In [ ]:
# 清空数据库（可选）
CLEAR = True  # 修改为 False 跳过
if CLEAR:
    with driver.session() as s:
        s.run("MATCH (n) DETACH DELETE n")
    print("✅ 数据库已清空")

In [ ]:
# 导入概念
print("导入概念节点...")
with driver.session() as s:
    for _, row in tqdm(concepts_df.iterrows(), total=len(concepts_df)):
        s.run("""
            MERGE (n:Concept {name: $name})
            SET n.category = $category, n.importance = $importance
        """, name=row['entity'], category=row['category'], importance=int(row['importance']))
print("✅ 概念导入完成")

In [ ]:
# 导入关系
print("导入关系...")
with driver.session() as s:
    for _, row in tqdm(relationships_df.iterrows(), total=len(relationships_df)):
        rel_type = row['edge'].replace(' ', '_').upper()
        s.run(f"""
            MATCH (a:Concept {{name: $head}})
            MATCH (b:Concept {{name: $tail}})
            MERGE (a)-[r:{rel_type}]->(b)
            SET r.weight = $weight
        """, head=row['node_1'], tail=row['node_2'], weight=float(row['weight']))
print("✅ 关系导入完成")

In [ ]:
# 验证
with driver.session() as s:
    nodes = s.run("MATCH (n) RETURN count(n) as c").single()['c']
    rels = s.run("MATCH ()-[r]->() RETURN count(r) as c").single()['c']
    print(f"\n✅ 导入成功！\n   节点: {nodes}\n   关系: {rels}")

In [ ]:
# 示例查询
with driver.session() as s:
    print("\n🌟 最重要的概念:")
    result = s.run("""
        MATCH (n:Concept) 
        RETURN n.name, n.importance, n.category 
        ORDER BY n.importance DESC LIMIT 10
    """)
    for r in result:
        print(f"  {r['n.name']} ({r['n.category']}, 重要性: {r['n.importance']})")

In [ ]:
driver.close()
print("\n🎉 全部完成！访问你的 Neo4j Browser 查看知识图谱")